In [ ]:
#Import all packages needed
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt #Visualisierungs Package (Vorlesung 5)
#Magic command, damit die Diagramme im Notebook angezeigt werden
%matplotlib inline 
import json  #Package um mit json files arbeiten zu können
import seaborn as sns #Visualisierungs Package (Vorlesung 5)

# Aufgabe

Quelle: *Diese Übung ist ein Teil aus einem Cornerstone Projekt aus dem Data Analyst Nanodegree von Udacity. Für diese Übung wurden einige Vereinfachungen vorgenommen.*

**Beschreibung**:

Habt Ihr jemals von dem Twitter-Account WeRateDogs gehört? Die Leute im Internet sind verrückt nach diesem Twitter-Account. Ich möchte Euch heute zeigen, warum. 

Heute hat WeRateDogs über 8,8 Millionen Follower. Das Schlüsselkonzept besteht darin, ein hübsches Bild der Doggos, Floofer, Pupper oder Puppos (so definieren sie das Entwicklungsstadium der Hunde) mit einer coolen Bildunterschrift und einer Bewertung mit einem Nenner von 10, aber einem Zähler, der alles sein kann, zu haben. In den meisten Fällen geht der Zähler deutlich über 10.


<div>
<img src="src/Picture 1.jpg" width="400"/>
</div>

**Inputdaten:**

Ingesamt haben wir 3 verschiedene Dateien, in denen sich aber leider verschiedene Fehler in den Daten befinden
- Das Twitter Archive der Tweets
- Eine Datei, in der die Rasse des Hundes vorausgesagt wurde
- Eine Datei, in der die Anzahl der Likes, Retweets etc pro Tweet steht


**Ziel**: 

Die Dateien wie unten vorgeschrieben zu bereinigen und zu einem Dataframe zusammenzufassen.

### Dokumentation der einzelnen Spalten der verschiedenen Quellen

Dokumentation Spalten ***Twitter_archive Dataframe***

Dieses Dataframe zeigt die Meta Daten jedes einzelnen Tweets für den betrachteten Zeitraum an.

- ***tweet_id*** ist die eineindeutige Twitter Post ID
- ***in_reply_to_status_id*** nicht relevant für uns
- ***in_reply_to_user_id*** nicht relevant für uns
- ***timestamp*** Zeitpunkt des posts
- ***source*** nicht relevant für uns
- ***text*** Tweet text
- ***retweeted_status_id*** ID wenn es ein retweet ist
- ***retweeted_status_user_id*** User ID wenn es ein retweet ist
- ***retweeted_status_user_timestamp*** User ID wenn es ein retweet ist
- ***expanded_urls*** URL des Tweets
- ***rating_numerator*** Zähler der Bewertung
- ***rating_denominator*** Nenner der Bewertung
- ***name*** Name des Hundes
- ***doggo*** Hundeentwicklungsstatus
- ***floofer*** Hundeentwicklungsstatus
- ***pupper*** Hundeentwicklungsstatus
- ***puppo*** Hundeentwicklungsstatus

### Dokumentation Spalten ***Image_prediction Dataframe***

Dieses DataFrame stammt aus einem Prediction Maschine Learning Algorithmus, welches an Hand der Bilder, versucht vorauszusagen, welche Rasse der Hund ist

- ***tweet_id*** ist die eineindeutige Twitter Post ID
- ***jpg_url*** URL des Bildes / der Bilder
- ***img_num*** Anzahl der Bilder
- ***p1/p2/p3*** Predicted Hunderasse
- ***p1_conf/p2_conf/p3_conf*** Wert aus der Klassifikation
- ***p1_dog/p2_dog/p3_dog*** Boolean ob der vorausgesagte p1/p2/p3 Wert ein Hund ist ja oder nein



### Dokumentation Spalten ***Tweet_json Dataframe***

Dieses DataFrame zeigt die numerischen Werte der einzelnen Tweets an.

- ***tweet_id*** ist die eineindeutige Twitter Post ID
- ***favorite_count*** Wie viele Favorites der Tweet bekommen hat
- ***retweet_count*** Wie oft der Tweet geretweeted wurde
- ***followers_count*** wie viele Follower es damals gab
- ***retweeted_status*** Ob der Tweet der Original Tweet war oder ein Retweet
- ***url*** URL des Tweets


## Gathering Data

Die nachfolgenden Zellen bis zum Start Der Übung dienen lediglich für Informationszwecke und werden vorgegeben, damit ihr euch schneller auf den wichtigten Teil der Übung, das Data Cleaning fokussieren könnt.

In [ ]:
#Twitter Archive (Dateiformat CSV) in einen DataFrame laden
twitter_archive = pd.read_csv("src/twitter-archive-enhanced.csv")

In [ ]:
#Bild-Vorhersagen (Image-Predictions - TSV file) in einen DataFrame laden
image_prediction = pd.read_csv('src/image-predictions.tsv', sep='\t' )

In [ ]:
'''
Herauslesen der interessanten Informationen aus JSON-Wörterbüchern in einer Txt-Datei
und in einen DataFrame tweets_json einfügen
'''

my_demo_list = []
with open('src/tweet_json.txt', encoding='utf-8') as json_file:  
    all_data = json.load(json_file)
    for each_dictionary in all_data:
        tweet_id = each_dictionary['id']
        whole_tweet = each_dictionary['text']
        only_url = whole_tweet[whole_tweet.find('https'):]
        favorite_count = each_dictionary['favorite_count']
        retweet_count = each_dictionary['retweet_count']
        followers_count = each_dictionary['user']['followers_count']
        retweeted_status = each_dictionary['retweeted_status'] = each_dictionary.get('retweeted_status', 'Original tweet')
        if retweeted_status == 'Original tweet':
            url = only_url
        else:
            retweeted_status = 'This is a retweet'
            url = 'This is a retweet'
            
        my_demo_list.append({'tweet_id': str(tweet_id),
                             'favorite_count': int(favorite_count),
                             'retweet_count': int(retweet_count),
                             'followers_count': int(followers_count),
                             'retweeted_status': retweeted_status,
                            })
        tweet_json = pd.DataFrame(my_demo_list, columns = ['tweet_id', 'favorite_count','retweet_count', 
                                                           'followers_count', 'retweeted_status', 'url'])
                             
                            

## Assess

In [ ]:
twitter_archive.info()

In [ ]:
### FINDINGS ###
# Es gibt sowohl Originalbewertungen und Retweets
# Zeitstempel ist in einem nicht korrekten Datenformat (hier Datetime)
# Fehlende Daten in den folgenden Spalten: in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, expanded_urls
# Hundeentwicklungsstufen sind in 4 Spalten

In [ ]:
twitter_archive.sample(5)

In [ ]:
### FINDINGS ###
# Hundestufen sind nicht korrekt angegeben
# Der Nenner der Bewertung ist zum Teil größer als 10

In [ ]:
sum(twitter_archive.name.unique()=='None')

In [ ]:
### FINDINGS ###
# Manchmal ist der Name des Hundes None

In [ ]:
image_prediction.info()

In [ ]:
image_prediction.sample(20)

In [ ]:
### FINDINGS ###
# p1, p2, p3 Hunderassen sind manchmal groß, manchmal klein geschrieben
# Die Spalten p1, p2 und p3 enthalten ungültige Daten, wie z. B. ein Vogelhaus, einen Dosenöffner oder einen Brustpanzer usw.

In [ ]:
tweet_json.info()

In [ ]:
### FINDINGS ###
# tweet_id ist ein String und keine Zahl

In [ ]:
tweet_json.sample(10)

In [ ]:
tweet_json.retweeted_status.unique()

In [ ]:
### FINDINGS ###
# Retweets in diesem df

In [ ]:
# Doppelte Bilder in den URLs enthalen?
image_prediction.jpg_url.value_counts()

In [ ]:
#Anzahl doppelter URLS
len(image_prediction.jpg_url) - image_prediction.jpg_url.nunique()

In [ ]:
### FINDINGS ###
# Es gibt doppelte Werte (an Hand der URL) 

In [ ]:
twitter_archive.rating_numerator.value_counts()

In [ ]:
with pd.option_context('max_colwidth', 300):
    display(twitter_archive[twitter_archive['text'].str.contains(r"(\d+\.\d*\/\d+)")]
            [['tweet_id', 'text', 'rating_numerator', 'rating_denominator']])

In [ ]:
### FINDINGS ###
# Wenn der Zähler eine Dezimalzahl ist wie z.B. 13.5 steht in der Spalte rating_numerator nur die ".5" 

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)


# Start der Übung

## Zusammenfassung der Beobachtungen aus erster Datenexploration 

##### **`twitter_archive`** 
 - Es gibt sowohl Originalbewertungen und Retweets
 - Es müssen Spalten gelöscht werden, die nicht für die Analyse verwendet werden
 - Hundestufen sind nicht korrekt angegeben
 - Zeitstempel ist in einem nicht korrekten Datenformat (hier Datetime)
 - Fehlende Daten in den folgenden Spalten: in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, expanded_urls
 - Der Nenner der Bewertung wird größer als 10
 - Wenn der Zähler eine Dezimalzahl ist wie z.B. 13.5 steht in der Spalte rating_numerator nur die ".5" 
 - Manchmal ist der Name des Hundes None
 - Hundeentwicklungsstufen sind in 4 Spalten
 - Alle drei Quellen sollen zum Schluss in einem DataFrame sein
 
  
 ##### **`tweet_json`**
 - tweet_id ist ein String und keine Zahl
 - fehlende Daten für die tweet_ids
 - Retweets in diesem df
 - Alle drei Quellen sollen zum Schluss in einem DataFrame sein
 
 
 ##### **`image_prediction`** 
 - p1, p2, p3 Hunderassen sind manchmal groß, manchmal klein geschrieben
 - Die Spalten p1, p2 und p3 enthalten ungültige Daten, wie z. B. ein Vogelhaus, einen Dosenöffner oder einen Brustpanzer usw.
 - Doppelte Zeilen
 - Alle drei Quellen sollen zum Schluss in einem DataFrame sein

## Aufgaben für euer Data Cleaning

**`twitter_archive`**</br>
1. Lösche alle Retweets und behalte die Originalbewertungen in `twitter_archive`
2. Lösche folgende Spalten in `twitter_archive`: 'source','in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls'
3. Konvertiere den Zeitstempel in `twitter_archive` als Datumsformat
4. Korrigiere den Zähler in `twitter_archive`. Wandle es erst als `float` um, und korrigiere die 6 falschen Werte manuell
5. Entferne alle Zeilen in `twitter_archive`, deren Nenner nicht 10 ist
6. Kombinieren alle Spalten der Dog Stages in eine gemeinsame Spalte in `twitter_archive`


**`tweet_json`**</br>
7. Wandle die tweet_id in ein `int` Format in `tweet_json`</br>
8. Lösche alle Retweets und behalte die Originalbewertungen in `tweet_json`
</br></br>
**`image_predictions`**</br>
9. Überabeite die Spalten der Hunderassen in `image_predictions` so, dass nur Kleinbuchstaben enthalten sind</br>
10. Lösche alle duplizierten Zeilen (ca. 66) mit Bild in `image_predictions`
</br></br>
**Übergreifend**</br>
11. Erstelle einen DataFrame aus den drei bereinigten DataFrames mit Namen `df`</br>
12. Nenne die Spalte text in tweet_text um im Dataframe `df` </br>
13. Outlier Rating_numerator erkennen und droppen, mit Hilfe der IQR

In [ ]:
'''
Wir erstellen im ersten Schritt immer eine "Hartcopy", 
auf deren Basis wir dann das Data Cleaning machen damit wir später immer Vergleichen können,
ob die Transformationen wirklich funktioniert haben
'''
twitter_archive_clean = twitter_archive.copy()
image_prediction_clean = image_prediction.copy()
tweet_json_clean = tweet_json.copy()

**1. Lösche alle Retweets und behalte die Originalbewertungen**

In [ ]:
#CODE


In [ ]:
#TEST


**2. Lösche folgende Spalten in `twitter_archive`: 'source','in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls'**

In [ ]:
#CODE


In [ ]:
#TEST


**3. Konvertiere den Zeitstempel in `twitter_archive` als Datumsformat**

In [ ]:
#CODE


In [ ]:
#TEST


**4. Korrigiere den Zähler in `twitter_archive`. Wandle es erst als `float` um, und korrigiere die 6 falschen Werte manuell**

In [ ]:
#CODE


#Manuelles Updaten der Nenner (numerators)


In [ ]:
#TEST
with pd.option_context('max_colwidth', 300):
    display(twitter_archive_clean[twitter_archive_clean['text'].str.contains(r"(\d+\.\d*\/\d+)")]
            [['tweet_id', 'text', 'rating_numerator', 'rating_denominator']])

**5. Entferne alle Zeilen in `tweet_archive`, deren Nenner nicht 10 ist**

In [ ]:
len(twitter_archive_clean[twitter_archive_clean["rating_denominator"] != 10])

In [ ]:
#CODE


In [ ]:
#TEST


**6. Kombinieren alle Spalten der Dog Stages in eine gemeinsame Spalte in `twitter_archive`**

In [ ]:
#CODE Melt the doggo, floofer, pupper and puppo columns to dogs and dogs_stage column


In [ ]:
#CODE drop dogs

#CODE Sort by dogs_stage then drop duplicated based on tweet_id except the last occurrence


In [ ]:
#TEST

**7. Wandle die tweet_id in ein `int` Format in `tweet_json`**

In [ ]:
#CODE


In [ ]:
#TEST


**8. Lösche alle Retweets und behalte die Originalbewertungen in `tweet_json`**

In [ ]:
#CODE
tweet_json_clean = tweet_json_clean[tweet_json_clean["retweeted_status"]=="Original tweet"]

In [ ]:
#TEST
tweet_json_clean.retweeted_status.value_counts()

**9. Überarbeite die Spalten der Hunderassen in `image_predictions` so, dass nur Kleinbuchstaben enthalten sind**

In [ ]:
#CODE


In [ ]:
#TEST


**10. Lösche alle duplizierten Zeilen (ca. 66) mit Bild in `image_predictions`**

In [ ]:
#CODE


In [ ]:
#TEST


**11. Erstelle einen DataFrame aus den drei bereinigten DataFrames mit Namen `df`**

In [ ]:
#CODE Erstellen eines neuen Dataframes df, welches twitter_archive_clean und image_prediction_clean merged

#keep rows that have picture (jpg_url)


In [ ]:
#TEST


In [ ]:
#CODE Erstellen eines neuen Dataframes df, welches df_twitter und tweet_json_clean merged


#TEST


**12. Nenne die Spalte text in tweet_text um im Dataframe `df`**

In [ ]:
#CODE


In [ ]:
#TEST


**13. Outlier Rating_numerator erkennen und droppen, mit Hilfe der IQR**

In [ ]:
# CODE - Detect Outlier


In [ ]:
# CODE - Delete Outliers


In [ ]:
# TEST


**FRAGE: Hat es Sinn gemacht, hier die vermeintlichen Outlier rauszuschmeißen?**

# ÜBUNG ENDE
![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## Speichern, Analysieren und Visualisieren der Daten

Ein kleiner Ausblick in die Übung 5, bei der es dann um die Visualisierung von Daten geht.

In [ ]:
#Speichern der twitter_dogs df in eine CSV Datei
df.to_csv('twitter_archive_master.csv', encoding='utf-8', index=False)

In [ ]:
df.info()

In [ ]:
dog_stages = df.groupby("dogs_stage")
dog_stages.describe()

In [ ]:
plt.figure(figsize=(10,10))
g = sns.boxplot(x='dogs_stage',y='retweet_count',data= df,palette='rainbow')
g.axes.set_title('Boxplot dog stages and retweet', fontsize=14);

Doggo stages has huge outliers, there needs to be more investigation to see why this happens at the doggo stages, besides that puppos have on average the most retweets followed by doggo and floofer

In [ ]:
df.plot(y='retweet_count', x='rating_numerator', kind='scatter')
plt.xlabel('Ratings')
plt.ylabel('Retweet Counts')
plt.title('Retweet Counts by Ratings Scatter Plot')

fig = plt.gcf()

As seen here, the better the rating the more retweets happened the most retweets happend between 12-14

In [ ]:
df_dog_type = df.groupby('p1').filter(lambda x: len(x) >= 25)

df_dog_type['p1'].value_counts().plot(kind = 'barh')
plt.title('Histogram of the Most Rated Dog Type')
plt.xlabel('Count')
plt.ylabel('Type of dog')

fig = plt.gcf() 
fig.savefig('output.png',bbox_inches='tight');

In [ ]:
dog_types_count = df_dog_type.p1.value_counts()
dog_types_avg_rating = df_dog_type.groupby('p1').rating_numerator.mean()

df_dog_type.p1.value_counts()

In [ ]:
plt.figure(figsize = (10, 10))
plt.plot(df.timestamp,df.favorite_count, marker = 'o', linestyle = '', ms = 2)
plt.title('Favorite tweet over year')
plt.xlabel('Date')
plt.ylabel('Fovorite Count')
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
plt.plot(df.timestamp,df.retweet_count, marker = 'o', linestyle = '', ms = 2)
plt.title('Retweets over year')
plt.xlabel('Date')
plt.ylabel('Retweets Count')
plt.show()